# Teaching Emotion Through Feature Matching Using Convolutional Neural Networks 



## Trial #1: VGG16 Model 

Note: Testing first without weights and then with weights 

In [1]:
# Core Dependencies (for all models)
import os
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning Dependencies
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Custom Dependencies (for VGG16 Model) 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout

## Trial #1: VGG16 w/ Random Weights

### Load and Pre-process FER2013 Dataset

In [2]:
# Define image parameters and paths
img_size = (48, 48)
batch_size = 64
dataset_test = 'FER2013/test'
dataset_train = 'FER2013/train'

### Create Data Generators 


In [3]:
# Train/Validation split from folder 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,  # Use part of training as validation
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# Create the generators
train_generator = train_datagen.flow_from_directory(
    'FER2013/train',
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',  # Training subset
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    'FER2013/train',
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  # Validation subset
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    'FER2013/test',
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 10239 images belonging to 2 classes.
Found 1806 images belonging to 2 classes.
Found 3021 images belonging to 2 classes.


### Define the Model

In [5]:
# Define VGG16 model
def build_vgg16(input_shape=(48, 48, 1), num_classes=7):
    base_model = VGG16(weights=None, include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

vgg16_model = build_vgg16()
vgg16_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,780,103 (56.38 MB)

 Trainable params: 14,780,103 (56.38 MB)

 Non-trainable params: 0 (0.00 B)

### Model Training (Random Weights)

In [6]:
# Train the model
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
]

history = vgg16_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30


/opt/anaconda3/envs/env1/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 7)

### Evaluate on Test Set 

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = vgg16_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Question: Was this training model successful? 

Answer: No, this model was not successful. While it was able to complete training without errors, the resulting test accuracy of 25% is significantly below the threshold for success in a 7-class classification task. Random guessing would yield approximately 14.3% accuracy (1 in 7), so although the model is performing slightly better than chance, it is not making consistently correct predictions. Furthermore, ideal loss after training should drop below 1.0, and ideally to < 0.5 for good accuracy. We recieved a 1.81 test loss.
* Accuracy: 25%
* Loss: 181%

### Plot Training History 

This is intial plot training history to show failure with using a model without any predefined weights.

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

## Trial #2: VGG16 with RGB Conversion 

In [ ]:
# Define image parameters and paths (adding rgb)
img_size = (48, 48)
batch_size = 64
rgb_size = (48, 48, 3)  # 3-channel input for VGG16 with ImageNet 

### Re-define the Data Generators (w/ RGB Conversion)

In [ ]:
def to_rgb(img):
    return np.repeat(img, 3, axis=2)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    dataset_test,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

### Load VGG16 Base w/ IMAGENET Weights

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=rgb_size)

# Freeze all base layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom classifier head 
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(7, activation='softmax')(x)

### Create Model (Test 2)

In [ ]:
# Create the model
model_2 = Model(inputs=base_model.input, outputs=output)

model_2.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_2.summary()

In [ ]:
# Train the model w/ ImageNet Weights 
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
]

history = model_2.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=callbacks
)

### Evaluate the Model 

In [ ]:
# Evaluate the model (test #2)
test_loss, test_accuracy = model_2.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

**Question: Was this training model successful?**

**Answer:** No, this training model was not successful. While it showed some learning progress, the overall performance fell short of expectations for a 7-class emotion classification task. The model used was VGG16 with ImageNet pretrained weights, which helped improve the accuracy to approximately 43%, but this is still well below an acceptable benchmark for real-world emotion recognition (typically 70–80%+). The loss remained high (~1.49 or 149%), indicating that the model’s confidence in its predictions was weak and often incorrect. Also, VGG16, even with ImageNet, is too shallow for nuanced emotions like fear vs surprise. It performs well on general image classification, but it struggles with similar emotions. Lastly, the model’s base layers were likely not fine-tuned deeply enough to adapt to facial features rather than general object recognition.
* Accuracy: 43% 
* Loss: almost 149% 